In [3]:
from bqplot import pyplot as plt
import bqplot as bq
import ipywidgets as widgets
from src.analysis import get_sac_onsets_offsets
from src.analysis import predict_possible_saccades
from src.imports import append_list_as_row
from src.imports import check_results_file
from src.imports import read_eye_data_mat
import math
import matplotlib
import numpy as np
from numpy import matlib
import mat73

In [42]:
# init some values
iTrial   = 0
curr_sac = 0
this_trial_save_flag = False

# check if csv file is present in the /results folder. if not, make it
check_results_file()

# get the data
x_matrix,y_matrix  = read_eye_data_mat() # these are trials by timepoints
trialtime          = len(x_matrix[iTrial])

# get the predicted saccade onsets and offsets
onsets   = np.array([])
offsets  = np.array([])


## functions are defined here ##
def new_trial(*ignore):
    global iTrial, trialtime
    # get the new length of the trial
    trialtime = len(x_matrix[iTrial])

    
    Line_eye_x.y =  x_matrix[iTrial] #- np.mean(x_matrix[ iTrial])
    Line_eye_y.y =  y_matrix[iTrial] #- np.mean(y_matrix[ iTrial])
    Line_eye_x.x =  np.arange(trialtime)
    Line_eye_y.x =  np.arange(trialtime)
    x_ax.scale.max =  trialtime
    
    y_ax.scale.min =  min([min(x_matrix[iTrial]),min(y_matrix[iTrial])])
    y_ax.scale.max =  max([max(x_matrix[iTrial]),max(y_matrix[iTrial])])


def next_trial(obj):
    global iTrial
    iTrial += 1
    new_trial()
    update()

    
def previous_trial(obj):
    global iTrial
    iTrial -= 1
    new_trial()
    update()

## Buttons and stuff
# trial buttons
next_trial_button = widgets.Button(description='Next trial',disabled=False,button_style='') 
next_trial_button.on_click(next_trial)
previous_trial_button = widgets.Button(description='Previous trial',disabled=False,button_style='') 
previous_trial_button.on_click(previous_trial)
    
    
# Plotting Stuff
animation_time = 200 # in ms

y_ax        = bq.Axis(label="x/y position (dva)",scale=bq.LinearScale(),orientation="vertical")
x_ax        = bq.Axis(label="time (ms)",scale=bq.LinearScale(min=1),orientation="horizontal")

Line_eye_x  = plt.plot(x=np.arange(trialtime),scales={'x':bq.LinearScale(),'y':bq.LinearScale()},colors = ['red'])
Line_eye_y  = plt.plot(x=np.arange(trialtime),scales={'x':bq.LinearScale(),'y':bq.LinearScale()},colors = ['green'])


eyetrace_fig = bq.Figure(animation_duration=animation_time,layout=widgets.Layout( width='auto'),axes=[x_ax,y_ax],
    marks=[Line_eye_x,Line_eye_y],fig_margin=dict(top=25,bottom=50,left=50,right=25))


button_box    = widgets.HBox(children=[previous_trial_button,next_trial_button],layout=widgets.Layout(display='flex',flex_flow='row',align_items='center',width='100%',justify_content='space-around'))


app = widgets.VBox(children=[button_box,eyetrace_fig],layout=widgets.Layout(border="solid 2px gray",width="1500px"))

# run app
new_trial()
app

